# Wide-and-Deep ML: Model Preparation

In this notebook, we train and evaluate the wide-and-deep collaborative filtering recommender using features engineered in the prior notebook.

## 1. Prepare the data

In [1]:
# import required libraries

import pandas as pd

In [15]:
# save models
train_df = pd.read_csv('../data/user_movie_interaction_train.csv')
val_df = pd.read_csv('../data/user_movie_interaction_val.csv')
test_df = pd.read_csv('../data/user_movie_interaction_train.csv')

In [16]:
train_df.head()

,Unnamed: 0,userId,movieId,rating,title,genres,genre_freq,user_genre_rating
0,58173,360,1235,0.8,Harold and Maude (1971),comedy drama romance,0.030536,0.024429
1,39649,258,3082,0.8,"World Is Not Enough, The (1999)",action adventure thriller,0.015289,0.012231
2,47232,347,4701,0.6,Rush Hour 2 (2001),action comedy,0.005520,0.003312
3,4820,471,1291,0.8,Indiana Jones and the Last Crusade (1989),action adventure,0.005702,0.004562
4,44925,294,1405,0.4,Beavis and Butt-Head Do America (1996),adventure animation comedy crime,0.000349,0.000140


In [19]:
# drop unnecessary columns
train_df.drop(['Unnamed: 0', 'movieId'], axis=1, inplace=True)
train_df.head()

,userId,rating,title,genres,genre_freq,user_genre_rating
0,360,0.8,Harold and Maude (1971),comedy drama romance,0.030536,0.024429
1,258,0.8,"World Is Not Enough, The (1999)",action adventure thriller,0.015289,0.012231
2,347,0.6,Rush Hour 2 (2001),action comedy,0.005520,0.003312
3,471,0.8,Indiana Jones and the Last Crusade (1989),action adventure,0.005702,0.004562
4,294,0.4,Beavis and Butt-Head Do America (1996),adventure animation comedy crime,0.000349,0.000140


### 1.1. Capture label and feature info

In [ ]:
LABEL_COL = 'title'

NUMERIC_COLS = [
    'rating',
    'genre_freq',
    'user_genre_rating'
]

CATEGORICAL_COL = [
    'userId',
    'genres'
]

HASH_BUCKET_SIZES = []

EMBEDDING_DIMENSIONS = []

## Future development
- use libraries like `PorterStemmer` that allows counter vectorization to find word associations in big paragraphs.
- `sklearn`'s `CountVectorizer` does the vectorization, and `cosine_similarity` computes how closely the vectorized words relate.